# Analyzing When and Where Influenza outbreak Occur in U.S


by Berthin Bitja

Note :
[How to install Homebrew on OS X?](https://stackoverflow.com/questions/20381128/how-to-install-homebrew-on-os-x)
[Old version X-code](https://stackoverflow.com/questions/10335747/how-to-download-xcode-dmg-or-xip-file)

In [23]:
options(warn = -1) #ignore warnings

# IMPORTANT: This assumes that all packages in "Rstart.R" are installed,
# and the fonts "Source Sans Pro" and "Open Sans Condensed Bold" are installed
# via extrafont. If ggplot2 charts fail to render, you may need to change/remove the theme call.


source("Rstart.R")
library(ggmap)



sessionInfo()

R version 3.2.3 (2015-12-10)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: OS X 10.12.6 (unknown)

locale:
[1] C

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] statebins_1.2.2    cdcfluview_0.5.1   ggmap_2.6.1        stringr_1.1.0     
 [5] digest_0.6.12      RColorBrewer_1.1-2 scales_0.4.0       extrafont_0.17    
 [9] ggplot2_2.1.0      dplyr_0.5.0        readr_0.2.2       

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.2-6        repr_0.12.0         reshape2_1.4.1     
 [4] purrr_0.2.3         lattice_0.20-33     V8_1.5             
 [7] colorspace_1.2-6    rlang_0.1.2         DBI_0.5-1          
[10] sp_1.2-2            uuid_0.1-2          jpeg_0.1-8         
[13] plyr_1.8.4          munsell_0.4.3       gtable_0.2.0       
[16] RgoogleMaps_1.2.0.7 mapproj_1.2-4       evaluate_0.10.1    
[19] labeling_0.3        curl_3.0            Rttf2pt1_1.3.4     
[22] proto_

# Processing the Data

In [4]:
#CDC information FLU
#devtools::install_github("hrbrmstr/cdcfluview")    
  

library(cdcfluview)
library(statebins)
# current verison
# packageVersion("cdcfluview")

#df.flu <- get_flu_data(region = "census", sub_region = 1:9, data_source = "all", years = 2008:2015)
#df.flu.census <- get_flu_data(region = "census", sub_region = 1:10, data_source = "ilinet", years = 2008:2015)

# I am looking at the ILI activity
df.state <- get_state_data(years=2010:2015)

In [5]:
#display the data 
#df.flu %>% head(10)
#sprintf("# of Rows in Dataframe: %s", nrow(df.flu))
#sprintf("Dataframe Size: %s", format(object.size(df.flu), units = "MB"))

In [6]:
#display the data 
df.state %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df.state))
sprintf("Dataframe Size: %s", format(object.size(df.state), units = "MB"))

statename,url,website,activity_level,activity_level_label,weekend,season,weeknumber
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,http://www.ct.gov/dph/cwp/view.asp?a=3136&q=410788,Flu Statistics,0,Insufficient Data,May-24-2014,2013-14,21
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Pages/Influenza.aspx,Puerto Rico Departamento de Salud,0,Insufficient Data,May-24-2014,2013-14,21
Virgin Islands,http://doh.vi.gov/,Influenza,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,http://www.ct.gov/dph/cwp/view.asp?a=3136&q=410788,Flu Statistics,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Pages/Influenza.aspx,Puerto Rico Departamento de Salud,0,Insufficient Data,May-31-2014,2013-14,22
Virgin Islands,http://doh.vi.gov/,Influenza,0,Insufficient Data,May-31-2014,2013-14,22
Idaho,http://www.healthandwelfare.idaho.gov/Health/DiseasesConditions/SeasonalInfluenzaFlu/tabid/327/Default.aspx,Influenza,0,Insufficient Data,Jun-07-2014,2013-14,23
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,Jun-07-2014,2013-14,23


[1] "# of Rows in Dataframe: 16668"

[1] "Dataframe Size: 0.9 Mb"

In [36]:
#display the data 
#df.flu.census %>% head(10)
#sprintf("# of Rows in Dataframe: %s", nrow(df.flu.census))
#sprintf("Dataframe Size: %s", format(object.size(df.flu.census), units = "MB"))

In [7]:
#columns = c("REGION TYPE","REGION","YEAR","WEEK","AGE 0-4","AGE 25-49","AGE 25-64","AGE 5-24","AGE 50-64","AGE 65","ILITOTAL","TOTAL PATIENTS")

columns = c("statename","activity_level","activity_level_label","weekend", "season", "weeknumber")

# select() require column indices that we find trough which() 

df <- df.state %>% select(which(names(df.state) %in% columns))

df %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df))
sprintf("Dataframe Size: %s", format(object.size(df), units = "MB"))

statename,activity_level,activity_level_label,weekend,season,weeknumber
District of Columbia,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,0,Insufficient Data,May-24-2014,2013-14,21
District of Columbia,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,0,Insufficient Data,May-24-2014,2013-14,21
Virgin Islands,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,0,Insufficient Data,May-31-2014,2013-14,22
Virgin Islands,0,Insufficient Data,May-31-2014,2013-14,22
Idaho,0,Insufficient Data,Jun-07-2014,2013-14,23
District of Columbia,0,Insufficient Data,Jun-07-2014,2013-14,23


[1] "# of Rows in Dataframe: 16668"

[1] "Dataframe Size: 0.7 Mb"

All-Caps to proper case. (see this [Stack Overflow question](http://stackoverflow.com/questions/15776732/how-to-convert-a-vector-of-strings-to-title-case))

In [8]:
proper_case <- function(x) {
    return (gsub("\\b([A-Z])([A-Z]+)", "\\U\\1\\L\\2" , x, perl = TRUE))
}

# df <- df %>% mutate(STATENAME = proper_case(STATENAME),
# ACTIVITYESTIMATE = proper_case(ACTIVITYESTIMATE),
#                 WEEKEND = proper_case(WEEKEND))

df %>% head(10)

statename,activity_level,activity_level_label,weekend,season,weeknumber
District of Columbia,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,0,Insufficient Data,May-24-2014,2013-14,21
District of Columbia,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,0,Insufficient Data,May-24-2014,2013-14,21
Virgin Islands,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,0,Insufficient Data,May-31-2014,2013-14,22
Virgin Islands,0,Insufficient Data,May-31-2014,2013-14,22
Idaho,0,Insufficient Data,Jun-07-2014,2013-14,23
District of Columbia,0,Insufficient Data,Jun-07-2014,2013-14,23


# Filtering the data

let's file the data frame by "Local Activity" to aggregate some interesting statistics

In [31]:
# grepl() is the best way to do in-text search
#df_flu <- df %>% filter(grepl("Local Activity", ACTIVITYESTIMATE))

df.flu.activity <- df %>% filter(activity_level > 0)

df.flu.activity %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df.flu.activity))
sprintf("Dataframe Size: %s", format(object.size(df.flu.activity), units = "MB"))

statename,activity_level,activity_level_label,weekend,season,weeknumber
New York City,1,Minimal,Oct-01-2016,2015-16,39
Florida,1,Minimal,Oct-01-2016,2015-16,39
Washington,1,Minimal,Oct-01-2016,2015-16,39
West Virginia,1,Minimal,Oct-01-2016,2015-16,39
Wisconsin,1,Minimal,Oct-01-2016,2015-16,39
Wyoming,1,Minimal,Oct-01-2016,2015-16,39
Ohio,1,Minimal,Oct-01-2016,2015-16,39
Oklahoma,1,Minimal,Oct-01-2016,2015-16,39
Oregon,1,Minimal,Oct-01-2016,2015-16,39
Pennsylvania,1,Minimal,Oct-01-2016,2015-16,39


[1] "# of Rows in Dataframe: 15878"

[1] "Dataframe Size: 0.6 Mb"

## Flue Over Time

In [32]:
 as.Date("May-17-2014","%b-%d-%Y")

[1] "2014-05-17"

In [11]:
df.flu.weekend <-df.flu.activity %>%
                    mutate(weekend = as.Date(weekend,"%b-%d-%Y")) %>% #convert to proper date for analysis
                    group_by(weekend) %>%
                    summarize(count = n()) %>%
                    arrange(weekend)

df.flu.weekend %>% head(10)

weekend,count
2010-10-09,52
2010-10-16,52
2010-10-23,52
2010-10-30,52
2010-11-06,52
2010-11-13,52
2010-11-20,52
2010-11-27,52
2010-12-04,52
2010-12-11,52


In [26]:
plot <- ggplot(df.flu.weekend, aes(x = weekend, y = count)) +
    geom_line(color = "#F2CA27", size = 0.1) +
    geom_smooth(color = "#1A1A1A") +
    fte_theme() +
    scale_x_date(breaks = date_breaks("2 years"), labels = date_format("%Y")) +
    labs(x = "Weekend of the obseration", y = "Count of weekly activity > 0 "
         , title = "Weekly ILI Activity in the US from 2015 - 2017")

In [27]:
max_save(plot, "US-ILI-when-1", "US CDC Fluview Data")

[How to Include image or picture in jupyter notebook](https://stackoverflow.com/questions/32370281/how-to-include-image-or-picture-in-jupyter-notebook)

![title](US-ILI-when-1.png)

##  ILI Activity Heatmap

Aggregate counts of ILI activity by Season and Months to create heat map.

-  [Extract Month and Year From Date in R](http://www.dummies.com/programming/r/how-to-extract-specific-elements-from-a-date-in-r/)

-  [How do I group my date variable into month/year in R?](https://stackoverflow.com/questions/33221425/how-do-i-group-my-date-variable-into-month-year-in-r)


In [60]:
# Returns the numeric hour component of a string formatted "HH:MM", e.g. "09:40" input returns 9
get_month <- function(x) {
    date <- as.Date(x, "%b-%d-%Y") #format the data to date
    month <- format(date, "%b") # return the month
    return (month)
}

df.flu.time <- df.flu.activity %>%
                    mutate(month = sapply(weekend, get_month)) %>%
                    group_by(month, season) %>% 
                    summarize(count = n())


df.flu.time %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df.flu.time))
sprintf("Dataframe Size: %s", format(object.size(df.flu.time), units = "MB"))

month,season,count
Apr,2010-11,256
Apr,2011-12,204
Apr,2012-13,205
Apr,2013-14,204
Apr,2014-15,208
Apr,2015-16,260
Aug,2010-11,194
Aug,2011-12,185
Aug,2012-13,233
Aug,2013-14,241


[1] "# of Rows in Dataframe: 72"

[1] "Dataframe Size: 0 Mb"

Reorder and format Factors.

In [61]:
month_format <- c("Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec")
season_format <- c("2010-11","2011-12","2012-13","2013-14","2014-15","2015-16")

df.flu.time$month <- factor(df.flu.time$month, level = rev(month_format))
df.flu.time$season <- factor(df.flu.time$season, level = rev(season_format))

df.flu.time %>% head(10)

month,season,count
Apr,2010-11,256
Apr,2011-12,204
Apr,2012-13,205
Apr,2013-14,204
Apr,2014-15,208
Apr,2015-16,260
Aug,2010-11,194
Aug,2011-12,185
Aug,2012-13,233
Aug,2013-14,241


In [84]:
plot <- ggplot(df.flu.time, aes(x = month, y = season, fill = count)) +
    geom_tile() +
    fte_theme() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.6),
          legend.title = element_blank(), legend.position="top", 
          legend.direction="horizontal", legend.key.width=unit(2, "cm"), 
          legend.key.height=unit(0.25, "cm"), legend.margin=unit(0.1,"cm"), 
          panel.margin=element_blank()) +
    labs(x = "Season of ILI activity ", y = "Month", 
         title = "# of ILI activity in U.S. from 2010 - 2015, by Season") +
    scale_fill_gradient(low = "white", high = "#27AE60", labels = comma)

max_save(plot, "US-ILI-when-2", "US CDC Fluview Data", w=6)

![title](US-ILI-when-2.png)

## Factor by Activity level



In [86]:
df.flu.type <- df.flu.activity %>%
                    group_by(activity_level_label) %>% 
                    summarize(count = n()) %>%
                    arrange(desc(count))

df.flu.type %>% head(20)
sprintf("# of Rows in Dataframe: %s", nrow(df.flu.type))

activity_level_label,count
Minimal,13422
Low,1007
High,861
Moderate,588


[1] "# of Rows in Dataframe: 4"